In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std

from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import mode

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 3.7 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100
from keras.utils import np_utils

In [ ]:
dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_train_sub.csv',header=None)

In [ ]:
dt[3000].value_counts()

0    2557
2     658
1     565
4     208
Name: 3000, dtype: int64

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way.csv',header=None)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_473_trial_2.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120701,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379,2
120702,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632,0
120703,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397,0
120704,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846,2


In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 14.0 MB/s 


In [ ]:
x_dt=transfer_set_with_tri_trained_sub_labeled_dt.drop([3000],axis=1)
y_dt=transfer_set_with_tri_trained_sub_labeled_dt[3000]

In [ ]:
y_dt_=np_utils.to_categorical(y_dt)

In [ ]:
y_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
x_dt.shape

(120706, 3000)

In [ ]:
x_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120701,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,0.004933,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379
120702,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.022106,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632
120703,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.000567,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397
120704,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.039062,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846


In [ ]:
x_dt_rshp=np.array(x_dt).reshape(120706,3000,1)

In [ ]:
x_dt_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.02463891],
        [ 0.03105487],
        [ 0.03884568],
        ...,
        [-0.0078992 ],
        [-0.01523173],
        [-0.02439739]],

       [[-0.01431516],
        [-0.05601893],
        [-0.06793429],
        ...,
        [-0.11238776],
        [-0.1059718 ],
        [-0.11284604]],

       [[-0.18449727],
        [-0.17117053],
        [-0.17164648],
        ...,
        [ 0.00921632],
        [ 0.01112014],
        [ 0.00112509]]])

In [ ]:
import numpy as np
np.random.seed(999)

import tensorflow
tensorflow.random.set_seed(999)

import random
#random.seed(999)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-hfcd95t6
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-hfcd95t6
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=6d33a860659ce1d259e62e852ed03958481f6c6106dab0baf2f8423a31ec7eac
  Stored in directory: /tmp/pip-ephem-wheel-cache-v4zir485/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
import keras_tuner as kt
optimizer=Adam(lr=1e-4)

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
      model=model_b(verbose=VBS)
      optimizer = Adam(lr=1e-4)
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
      optimizer = Adam(lr=1e-4)
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            x_dt_rshp, y_dt_,
            epochs=hp.Choice("epochs", [40, 50, 60, 70])
        )

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    #directory="my_dir",
    #project_name="tune_hypermodel",
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
tuner.search(x_dt_rshp, y_dt_)

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
epochs: 70
Score: 0.05816647596657276
Trial summary
Hyperparameters:
epochs: 50
Score: 0.07576893270015717
Trial summary
Hyperparameters:
epochs: 40
Score: 0.08691780269145966


tuner.search()

Trial 1:

Trial 3 Complete [01h 11m 41s]

loss: 0.05742859095335007

Best loss So Far: 0.05742859095335007

Total elapsed time: 02h 41m 55s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary()

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7ff72828d950>

Trial summary

Hyperparameters:

epochs: 70

Score: 0.05742859095335007

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07531780749559402

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08668727427721024

Trial 2

tuner.serach()

Trial 3 Complete [00h 41m 01s]

loss: 0.08829596266150475

Best loss So Far: 0.06591451540589333

Total elapsed time: 02h 20m 29s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary():

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f470d36c790>

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06591451540589333

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08692312240600586

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08829596266150475

Trial 3:

Tuner.results_summary():

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7fc7fc22be90>

Trial summary

Hyperparameters:

epochs: 70

Score: 0.05816647596657276

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07576893270015717

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08691780269145966

Training the teacher model for 40 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
teacher_40_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_teacher_model_40_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_40_epochs_dir = os.path.dirname(checkpoint_teacher_model_40_epochs)

In [ ]:
cp_callback_teacher_model_40_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_40_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=40)

In [ ]:
teacher_40_epochs.fit(x_dt_rshp, y_dt_, epochs=40, callbacks=[cp_callback_teacher_model_40_epochs])

Epoch 1/40
3773/3773 [==============================] - 44s 7ms/step - loss: 0.6679 - accuracy: 0.7315
Epoch 2/40
3773/3773 [==============================] - 30s 8ms/step - loss: 0.3830 - accuracy: 0.8499
Epoch 3/40
3773/3773 [==============================] - 28s 7ms/step - loss: 0.3174 - accuracy: 0.8746
Epoch 4/40
3773/3773 [==============================] - 28s 7ms/step - loss: 0.2790 - accuracy: 0.8901
Epoch 5/40
3773/3773 [==============================] - 27s 7ms/step - loss: 0.2483 - accuracy: 0.9018
Epoch 6/40
3773/3773 [==============================] - 28s 7ms/step - loss: 0.2271 - accuracy: 0.9096
Epoch 7/40
3773/3773 [==============================] - 28s 7ms/step - loss: 0.2100 - accuracy: 0.9165
Epoch 8/40
3773/3773 [==============================] - 28s 7ms/step - loss: 0.1988 - accuracy: 0.9219
Epoch 9/40
3773/3773 [==============================] - 28s 7ms/step - loss: 0.1899 - accuracy: 0.9246
Epoch 10/40
3773/3773 [==============================] - 28s 7ms/step - l

In [ ]:
latest_teacher_40_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_40_epochs_dir)

In [ ]:
print(latest_teacher_40_epochs)
print(checkpoint_teacher_model_40_epochs)

/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt
/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt


In [ ]:
sub_test_data=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
sub_test_data.shape

(997, 3001)

In [ ]:
x_test_dt=sub_test_data.drop([3000],axis=1)
y_test_dt=sub_test_data[3000]

In [ ]:
y_test_dt_=np_utils.to_categorical(y_test_dt)

In [ ]:
y_test_dt_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
x_test_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.045043,-0.060749,-0.036475,-0.148801,-0.090258,-0.033144,-0.089782,-0.091686,-0.106441,-0.140709,...,0.025399,0.018735,-0.010774,-0.001731,0.049197,0.061095,0.037774,0.085845,0.038250,0.019211
1,0.047095,0.051678,0.048011,0.055344,0.047095,0.053969,0.051219,0.007224,0.011807,0.089257,...,-0.113763,0.085591,-0.123845,0.060843,-0.138510,-0.149050,0.010890,-0.095431,-0.128428,0.000808
2,0.039304,0.029680,0.024639,0.030597,0.038387,0.036554,0.050761,0.059468,0.050761,0.046178,...,-0.147676,-0.126136,-0.138968,-0.127053,-0.122928,-0.116512,-0.115137,-0.115596,-0.124761,-0.114679
3,-0.050278,-0.048850,-0.049326,-0.055038,-0.052658,-0.051230,-0.035999,-0.028384,-0.012678,-0.006966,...,-0.003159,-0.013154,-0.013630,-0.010298,-0.006966,-0.005062,-0.004110,-0.013154,-0.029812,-0.042663
4,-0.077558,-0.068851,-0.067934,-0.068851,-0.067934,-0.072059,-0.071142,-0.068851,-0.071142,-0.060602,...,-0.027605,-0.026231,-0.032188,-0.032188,-0.040437,-0.044562,-0.038604,-0.036313,-0.038146,-0.016607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,-0.021721,-0.017437,-0.023625,-0.030288,-0.035999,-0.023625,-0.003634,0.014452,0.014928,0.011120,...,0.004933,-0.001255,0.003505,-0.003634,-0.007918,-0.013154,-0.012678,-0.020769,-0.018389,-0.014105
993,0.015015,0.013640,0.007224,0.016848,0.022806,0.027847,0.031971,0.036554,0.024181,0.009516,...,0.017306,0.032430,0.026472,0.007682,0.004474,0.023722,0.042512,0.027847,0.027847,0.005391
994,-0.006983,-0.001483,-0.001942,-0.007899,0.000350,0.002183,0.001266,-0.006983,0.000350,0.004933,...,0.029222,0.007682,0.011349,0.020973,0.013640,0.007224,-0.001025,-0.006983,-0.011107,-0.010191
995,0.261571,0.240490,0.223076,0.205202,0.196953,0.173123,0.160291,0.153416,0.144251,0.119045,...,0.034263,0.040679,0.059468,0.064968,0.064510,0.073675,0.075967,0.070926,0.055344,0.041137


In [ ]:
x_test_dt.shape

(997, 3000)

In [ ]:
x_test_dt_rshp=np.array(x_test_dt).reshape(997,3000,1)

In [ ]:
x_test_dt_rshp

array([[[-0.04504252],
        [-0.06074903],
        [-0.03647533],
        ...,
        [ 0.08584504],
        [ 0.03824956],
        [ 0.01921137]],

       [[ 0.04709478],
        [ 0.05167761],
        [ 0.04801135],
        ...,
        [-0.09543128],
        [-0.12842767],
        [ 0.00080818]],

       [[ 0.03930397],
        [ 0.02968002],
        [ 0.02463891],
        ...,
        [-0.11559574],
        [-0.12476141],
        [-0.11467918]],

       ...,

       [[-0.00698263],
        [-0.00148324],
        [-0.00194152],
        ...,
        [-0.00698263],
        [-0.01110718],
        [-0.01019062]],

       [[ 0.2615713 ],
        [ 0.24049027],
        [ 0.22307551],
        ...,
        [ 0.07092551],
        [ 0.05534388],
        [ 0.0411371 ]],

       [[-0.0078992 ],
        [-0.00102495],
        [-0.00469122],
        ...,
        [-0.01019062],
        [-0.00377465],
        [-0.01248203]]])

In [ ]:
accuracy_teacher_model_40_epochs = teacher_40_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 1s 6ms/step - loss: 1.2534 - accuracy: 0.7884


In [ ]:
preds = teacher_40_epochs.predict(x_test_dt_rshp)

In [ ]:
preds_fin=np.argmax(preds,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.90822785, 0.1496063 , 0.87428571, 0.63492063])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1 = f1_score(y_test_dt, preds_fin, average="macro")
prec = precision_score(y_test_dt, preds_fin, average="macro")
recall = recall_score(y_test_dt, preds_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec)) 
    print("Recall: {}".format(recall))
    print("F1 score: {}".format(f1))

    report = classification_report(y_test_dt, preds_fin, digits=6)
    print(report)

Precision: 0.5898636888324631
Recall: 0.6417601241300533
F1 score: 0.5816602371809771
              precision    recall  f1-score   support

           0   0.972881  0.908228  0.939444       632
           1   0.404255  0.149606  0.218391       127
           2   0.698630  0.874286  0.776650       175
           4   0.283688  0.634921  0.392157        63

    accuracy                       0.788365       997
   macro avg   0.589864  0.641760  0.581660       997
weighted avg   0.808760  0.788365  0.784437       997



Training the teacher model for 50 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_50_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_50_epochs_dir = os.path.dirname(checkpoint_teacher_model_50_epochs)

In [ ]:
cp_callback_teacher_model_50_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_50_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=50)

In [ ]:
teacher_50_epochs=model_b(verbose=VBS)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
teacher_50_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_50_epochs.fit(x_dt_rshp, y_dt_, epochs=50, callbacks=[cp_callback_teacher_model_50_epochs])

Epoch 1/50
3773/3773 [==============================] - 55s 10ms/step - loss: 0.6916 - accuracy: 0.7215
Epoch 2/50
3773/3773 [==============================] - 37s 10ms/step - loss: 0.3945 - accuracy: 0.8451
Epoch 3/50
3773/3773 [==============================] - 37s 10ms/step - loss: 0.3197 - accuracy: 0.8739
Epoch 4/50
3773/3773 [==============================] - 37s 10ms/step - loss: 0.2779 - accuracy: 0.8906
Epoch 5/50
3773/3773 [==============================] - 37s 10ms/step - loss: 0.2471 - accuracy: 0.9026
Epoch 6/50
3773/3773 [==============================] - 37s 10ms/step - loss: 0.2222 - accuracy: 0.9124
Epoch 7/50
3773/3773 [==============================] - 37s 10ms/step - loss: 0.2055 - accuracy: 0.9184
Epoch 8/50
3773/3773 [==============================] - 38s 10ms/step - loss: 0.1928 - accuracy: 0.9233
Epoch 9/50
3773/3773 [==============================] - 37s 10ms/step - loss: 0.1840 - accuracy: 0.9263
Epoch 10/50
3773/3773 [==============================] - 37s 10m

In [ ]:
latest_teacher_50_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_50_epochs_dir)

In [ ]:
accuracy_teacher_model_50_epochs = teacher_50_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 1s 7ms/step - loss: 1.2691 - accuracy: 0.7914


In [ ]:
preds = teacher_50_epochs.predict(x_test_dt_rshp)

In [ ]:
preds_fin=np.argmax(preds,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.92088608, 0.14173228, 0.88      , 0.55555556])

Training the teacher model for 60 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_60_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_60_epochs_dir = os.path.dirname(checkpoint_teacher_model_60_epochs)

In [ ]:
cp_callback_teacher_model_60_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_60_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=60)

In [ ]:
teacher_60_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_60_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_60_epochs.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])

Epoch 1/60
3773/3773 [==============================] - 47s 8ms/step - loss: 0.6699 - accuracy: 0.7295
Epoch 2/60
3773/3773 [==============================] - 30s 8ms/step - loss: 0.3873 - accuracy: 0.8475
Epoch 3/60
3773/3773 [==============================] - 30s 8ms/step - loss: 0.3216 - accuracy: 0.8730
Epoch 4/60
3773/3773 [==============================] - 31s 8ms/step - loss: 0.2837 - accuracy: 0.8883
Epoch 5/60
3773/3773 [==============================] - 30s 8ms/step - loss: 0.2539 - accuracy: 0.9001
Epoch 6/60
3773/3773 [==============================] - 30s 8ms/step - loss: 0.2306 - accuracy: 0.9088
Epoch 7/60
3773/3773 [==============================] - 30s 8ms/step - loss: 0.2129 - accuracy: 0.9157
Epoch 8/60
3773/3773 [==============================] - 31s 8ms/step - loss: 0.2008 - accuracy: 0.9216
Epoch 9/60
3773/3773 [==============================] - 31s 8ms/step - loss: 0.1898 - accuracy: 0.9250
Epoch 10/60
3773/3773 [==============================] - 30s 8ms/step - l

In [ ]:
latest_teacher_60_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_60_epochs_dir)

In [ ]:
accuracy_teacher_model_60_epochs = teacher_60_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 1s 6ms/step - loss: 1.3755 - accuracy: 0.7864


Training the teacher model for 70 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_70_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_70_epochs_dir = os.path.dirname(checkpoint_teacher_model_70_epochs)

In [ ]:
cp_callback_teacher_model_70_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_70_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=70)

In [ ]:
teacher_70_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_70_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_70_epochs.fit(x_dt_rshp, y_dt_, epochs=70, callbacks=[cp_callback_teacher_model_70_epochs])

Epoch 1/70
3773/3773 [==============================] - 55s 10ms/step - loss: 0.6718 - accuracy: 0.7286
Epoch 2/70
3773/3773 [==============================] - 37s 10ms/step - loss: 0.3872 - accuracy: 0.8484
Epoch 3/70
3773/3773 [==============================] - 37s 10ms/step - loss: 0.3171 - accuracy: 0.8742
Epoch 4/70
3773/3773 [==============================] - 37s 10ms/step - loss: 0.2763 - accuracy: 0.8920
Epoch 5/70
3773/3773 [==============================] - 37s 10ms/step - loss: 0.2463 - accuracy: 0.9026
Epoch 6/70
3773/3773 [==============================] - 37s 10ms/step - loss: 0.2234 - accuracy: 0.9113
Epoch 7/70
3773/3773 [==============================] - 38s 10ms/step - loss: 0.2063 - accuracy: 0.9185
Epoch 8/70
3773/3773 [==============================] - 38s 10ms/step - loss: 0.1966 - accuracy: 0.9222
Epoch 9/70
3773/3773 [==============================] - 38s 10ms/step - loss: 0.1855 - accuracy: 0.9270
Epoch 10/70
3773/3773 [==============================] - 38s 10m

In [ ]:
latest_teacher_70_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_70_epochs_dir)

In [ ]:
accuracy_teacher_model_70_epochs = teacher_70_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 2s 7ms/step - loss: 1.3937 - accuracy: 0.7944


KD using 70 epochs teacher, 80 epochs student, alpha: 0.9, temp: 5-->Trial 1

KD using 60 epochs teacher, 70 epochs student , alpha: 0.9, temp: 5 ---->trial 2

KD using 70 epochs teacher, 80 epochs student , alpha: 0.9, temp: 5 ---->trial 3

In [ ]:
teacher_model=model_b(verbose=VBS) #40 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 12s 8ms/step - loss: 1.2534 - accuracy: 0.7884


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.90822785, 0.1496063 , 0.87428571, 0.63492063])

In [ ]:
print(classification_report(y_test_dt, preds_teacher_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.972881  0.908228  0.939444       632
           1   0.404255  0.149606  0.218391       127
           2   0.698630  0.874286  0.776650       175
           4   0.283688  0.634921  0.392157        63

    accuracy                       0.788365       997
   macro avg   0.589864  0.641760  0.581660       997
weighted avg   0.808760  0.788365  0.784437       997



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=50)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/50
3773/3773 [==============================] - 36s 8ms/step - categorical_accuracy: 0.6854 - student_loss: 0.7690 - distillation_loss: 0.0016
Epoch 2/50
3773/3773 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7669 - student_loss: 0.5640 - distillation_loss: 0.0012
Epoch 3/50
3773/3773 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7830 - student_loss: 0.5243 - distillation_loss: 0.0011
Epoch 4/50
3773/3773 [==============================] - 32s 8ms/step - categorical_accuracy: 0.7957 - student_loss: 0.4996 - distillation_loss: 0.0010
Epoch 5/50
3773/3773 [==============================] - 32s 8ms/step - categorical_accuracy: 0.8048 - student_loss: 0.4789 - distillation_loss: 9.9555e-04
Epoch 6/50
3773/3773 [==============================] - 31s 8ms/step - categorical_accuracy: 0.8152 - student_loss: 0.4556 - distillation_loss: 9.3949e-04
Epoch 7/50
3773/3773 [==============================] - 31s 8ms/step - categorical_acc

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 0s 3ms/step - categorical_accuracy: 0.8004 - student_loss: 1.1084


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.967374  0.938291  0.952610       632
           1   0.428571  0.094488  0.154839       127
           2   0.639676  0.902857  0.748815       175
           4   0.321101  0.555556  0.406977        63

    accuracy                       0.800401       997
   macro avg   0.589181  0.622798  0.565810       997
weighted avg   0.800382  0.800401  0.780739       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.93829114, 0.09448819, 0.90285714, 0.55555556])

In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
#50 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #50 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 11s 6ms/step - loss: 1.2691 - accuracy: 0.7914


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.92088608, 0.14173228, 0.88      , 0.55555556])

In [ ]:
print(classification_report(y_test_dt, preds_teacher_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.973244  0.920886  0.946341       632
           1   0.352941  0.141732  0.202247       127
           2   0.706422  0.880000  0.783715       175
           4   0.269231  0.555556  0.362694        63

    accuracy                       0.791374       997
   macro avg   0.575460  0.624543  0.573749       997
weighted avg   0.802908  0.791374  0.786131       997



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 1497, 4)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_8 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 1497, 4)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=60)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/60
3773/3773 [==============================] - 27s 6ms/step - categorical_accuracy: 0.7058 - student_loss: 0.7334 - distillation_loss: 0.0015
Epoch 2/60
3773/3773 [==============================] - 23s 6ms/step - categorical_accuracy: 0.7773 - student_loss: 0.5406 - distillation_loss: 0.0012
Epoch 3/60
3773/3773 [==============================] - 22s 6ms/step - categorical_accuracy: 0.7912 - student_loss: 0.5074 - distillation_loss: 0.0011
Epoch 4/60
3773/3773 [==============================] - 22s 6ms/step - categorical_accuracy: 0.8020 - student_loss: 0.4833 - distillation_loss: 0.0010
Epoch 5/60
3773/3773 [==============================] - 22s 6ms/step - categorical_accuracy: 0.8142 - student_loss: 0.4604 - distillation_loss: 9.8578e-04
Epoch 6/60
3773/3773 [==============================] - 22s 6ms/step - categorical_accuracy: 0.8218 - student_loss: 0.4406 - distillation_loss: 9.4275e-04
Epoch 7/60
3773/3773 [==============================] - 22s 6ms/step - categorical_acc

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7713 - student_loss: 1.2729


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.969543  0.906646  0.937040       632
           1   0.178571  0.039370  0.064516       127
           2   0.660714  0.845714  0.741855       175
           4   0.279221  0.682540  0.396313        63

    accuracy                       0.771314       997
   macro avg   0.522012  0.618567  0.534931       997
weighted avg   0.770959  0.771314  0.757468       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.90664557, 0.03937008, 0.84571429, 0.68253968])

In [ ]:
#60 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 13s 6ms/step - loss: 1.3755 - accuracy: 0.7864


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.91139241, 0.13385827, 0.88      , 0.58730159])

In [ ]:
print(classification_report(y_test_dt, preds_teacher_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.972973  0.911392  0.941176       632
           1   0.369565  0.133858  0.196532       127
           2   0.696833  0.880000  0.777778       175
           4   0.268116  0.587302  0.368159        63

    accuracy                       0.786359       997
   macro avg   0.576872  0.628138  0.570911       997
weighted avg   0.803100  0.786359  0.781433       997



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3773/3773 [==============================] - 36s 8ms/step - categorical_accuracy: 0.7022 - student_loss: 0.7341 - distillation_loss: 0.0016
Epoch 2/70
3773/3773 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7732 - student_loss: 0.5466 - distillation_loss: 0.0012
Epoch 3/70
3773/3773 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7849 - student_loss: 0.5197 - distillation_loss: 0.0012
Epoch 4/70
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7928 - student_loss: 0.5008 - distillation_loss: 0.0011
Epoch 5/70
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8037 - student_loss: 0.4777 - distillation_loss: 0.0011
Epoch 6/70
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8179 - student_loss: 0.4494 - distillation_loss: 9.8553e-04
Epoch 7/70
3773/3773 [==============================] - 30s 8ms/step - categorical_accurac

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 0s 4ms/step - categorical_accuracy: 0.7844 - student_loss: 1.4879


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.970443  0.935127  0.952458       632
           1   0.365854  0.118110  0.178571       127
           2   0.763006  0.754286  0.758621       175
           4   0.252874  0.698413  0.371308        63

    accuracy                       0.784353       997
   macro avg   0.588044  0.626484  0.565239       997
weighted avg   0.811676  0.784353  0.783132       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.93512658, 0.11811024, 0.75428571, 0.6984127 ])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm.diagonal()

array([0.93512658, 0.11811024, 0.75428571, 0.6984127 ])

In [ ]:
#70 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #70 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 12s 6ms/step - loss: 1.3937 - accuracy: 0.7944


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.92563291, 0.13385827, 0.88      , 0.57142857])

In [ ]:
print(classification_report(y_test_dt, preds_teacher_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.973378  0.925633  0.948905       632
           1   0.369565  0.133858  0.196532       127
           2   0.716279  0.880000  0.789744       175
           4   0.266667  0.571429  0.363636        63

    accuracy                       0.794383       997
   macro avg   0.581472  0.627730  0.574704       997
weighted avg   0.806678  0.794383  0.788146       997



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=80)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/80
3773/3773 [==============================] - 34s 8ms/step - categorical_accuracy: 0.6779 - student_loss: 0.8174 - distillation_loss: 0.0018
Epoch 2/80
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7608 - student_loss: 0.5787 - distillation_loss: 0.0013
Epoch 3/80
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7814 - student_loss: 0.5287 - distillation_loss: 0.0012
Epoch 4/80
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7943 - student_loss: 0.5003 - distillation_loss: 0.0011
Epoch 5/80
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8094 - student_loss: 0.4692 - distillation_loss: 0.0010
Epoch 6/80
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8186 - student_loss: 0.4472 - distillation_loss: 9.7614e-04
Epoch 7/80
3773/3773 [==============================] - 30s 8ms/step - categorical_accurac

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7844 - student_loss: 1.0826


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.971667  0.922468  0.946429       632
           1   0.312500  0.078740  0.125786       127
           2   0.685185  0.845714  0.757033       175
           4   0.275168  0.650794  0.386792        63

    accuracy                       0.784353       997
   macro avg   0.561130  0.624429  0.554010       997
weighted avg   0.793404  0.784353  0.773286       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.92246835, 0.07874016, 0.84571429, 0.65079365])

In [ ]:
#50 epochs teacher model

#alpha=0.9, temp=4

#50 epochs: 82.41
#60 epochs: 84.53
#70 epochs: 84.04

#alpha=0.9, temp=5

#50 epochs: 81.43
#60 epochs: 83.88
#70 epochs: 

#alpha=0.9, temp=3

#50 epochs: 83.88
#60 epochs: 83.06


Trying out diff teachers and different students for perspective(the teacher who's epochs had been tuned)

In [ ]:
final_teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
final_teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_teacher_model.fit(x_dt_rshp, y_dt_, epochs=60)

Epoch 1/60
3773/3773 [==============================] - 48s 8ms/step - loss: 0.6674 - accuracy: 0.7316
Epoch 2/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.3828 - accuracy: 0.8497
Epoch 3/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.3181 - accuracy: 0.8761
Epoch 4/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.2783 - accuracy: 0.8907
Epoch 5/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.2462 - accuracy: 0.9030
Epoch 6/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.2234 - accuracy: 0.9121
Epoch 7/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.2064 - accuracy: 0.9188
Epoch 8/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.1938 - accuracy: 0.9239
Epoch 9/60
3773/3773 [==============================] - 29s 8ms/step - loss: 0.1828 - accuracy: 0.9269
Epoch 10/60
3773/3773 [==============================] - 29s 8ms/step - l

In [ ]:
accuracy_final_teacher_model = final_teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 1s 6ms/step - loss: 1.4617 - accuracy: 0.7753


In [ ]:
preds_final_teacher_model = final_teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_teacher_model_fin=np.argmax(preds_final_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_teacher = confusion_matrix(y_test_dt, preds_final_teacher_model_fin)
matrix_final_teacher.diagonal()/matrix_final_teacher.sum(axis=1)

array([0.90506329, 0.11023622, 0.86857143, 0.55555556])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_teacher = f1_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
prec_final_teacher = precision_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
recall_final_teacher = recall_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_teacher)) 
    print("Recall: {}".format(recall_final_teacher))
    print("F1 score: {}".format(f1_final_teacher))

    report_final_teacher = classification_report(y_test_dt, preds_final_teacher_model_fin, digits=6)
    print(report_final_teacher)

Precision: 0.5551027158170014
Recall: 0.6098566239346663
F1 score: 0.5517491265954609
              precision    recall  f1-score   support

           0   0.972789  0.905063  0.937705       632
           1   0.318182  0.110236  0.163743       127
           2   0.684685  0.868571  0.765743       175
           4   0.244755  0.555556  0.339806        63

    accuracy                       0.775326       997
   macro avg   0.555103  0.609857  0.551749       997
weighted avg   0.792830  0.775326  0.771151       997



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
#student_model=get_model_cnn()
final_student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=final_student_model, teacher=final_teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=1.0,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3773/3773 [==============================] - 32s 8ms/step - categorical_accuracy: 0.6896 - student_loss: 0.7606 - distillation_loss: 0.0016
Epoch 2/70
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7724 - student_loss: 0.5544 - distillation_loss: 0.0012
Epoch 3/70
3773/3773 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7892 - student_loss: 0.5130 - distillation_loss: 0.0011
Epoch 4/70
3773/3773 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8034 - student_loss: 0.4833 - distillation_loss: 0.0010
Epoch 5/70
3773/3773 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8139 - student_loss: 0.4605 - distillation_loss: 9.9406e-04
Epoch 6/70
3773/3773 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8237 - student_loss: 0.4357 - distillation_loss: 9.3697e-04
Epoch 7/70
3773/3773 [==============================] - 29s 8ms/step - categorical_acc

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7813 - student_loss: 1.0607


In [ ]:
preds_final_student_model = final_student_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_student_model_fin=np.argmax(preds_final_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_student = confusion_matrix(y_test_dt, preds_final_student_model_fin)
matrix_final_student.diagonal()/matrix_final_student.sum(axis=1)

array([0.92405063, 0.1496063 , 0.82857143, 0.49206349])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_student = f1_score(y_test_dt, preds_final_student_model_fin, average="macro")
prec_final_student = precision_score(y_test_dt, preds_final_student_model_fin, average="macro")
recall_final_student = recall_score(y_test_dt, preds_final_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_student)) 
    print("Recall: {}".format(recall_final_student))
    print("F1 score: {}".format(f1_final_student))

    report_final_student = classification_report(y_test_dt, preds_final_student_model_fin, digits=6)
    print(report_final_student)

Precision: 0.5590992708136313
Recall: 0.5985729631897279
F1 score: 0.5594939257388387
              precision    recall  f1-score   support

           0   0.966887  0.924051  0.944984       632
           1   0.311475  0.149606  0.202128       127
           2   0.721393  0.828571  0.771277       175
           4   0.236641  0.492063  0.319588        63

    accuracy                       0.781344       997
   macro avg   0.559099  0.598573  0.559494       997
weighted avg   0.794165  0.781344  0.780348       997

